In [1]:
import numpy as np
import tensorflow as tf

In [2]:
vocab_size = 80000
word_embed_size = 128
batch_size = 50
num_classes = 2
filter_num = 64
learning_rate = 0.01
training_steps = 10
print_loss_every = 2
filter_sizes = [3, 4, 5]

In [3]:
train = np.loadtxt('../data/train_data.txt', dtype=int)
test = np.loadtxt('../data/test_data.txt', dtype=int)
train_shuffle_idx = np.random.permutation(train.shape[0])
test_shuffle_idx = np.random.permutation(test.shape[0])
train = train[train_shuffle_idx]
test = test[test_shuffle_idx]
x_train = train[:, :-1]
y_train = train[:, -1:].reshape((-1,))
x_test = test[:, :-1]
y_test = test[:, -1:].reshape((-1,))
sequence_length = x_train.shape[1]
dataset_size = train.shape[0]

In [4]:
tf.reset_default_graph()

In [5]:
input_x = tf.placeholder(tf.int32, shape=[None, sequence_length], name='input_x')
input_y = tf.placeholder(tf.int32, shape=[None, ], name='input_y')
keep_proba = tf.placeholder(tf.float32, shape=None, name='keep_proba')

In [6]:
word_embedding = tf.get_variable('word_embedding', 
                    [vocab_size, word_embed_size],
                    tf.float32, 
                    tf.random_normal_initializer())
embeds = tf.nn.embedding_lookup(word_embedding, input_x)
embeds_expanded = tf.expand_dims(embeds, -1)

In [7]:
f3, f4, f5 = filter_sizes
pooled_outputs = []

In [8]:
with tf.name_scope(f'conv-maxpool-{f3}'):
    filter_shape3 = [f3, word_embed_size, 1, filter_num]
    W3 = tf.get_variable(f"W-{f3}", 
                        filter_shape3, 
                        initializer=tf.truncated_normal_initializer(stddev=0.1))
    b3 = tf.get_variable(f"b-{f3}", [filter_num], 
                        initializer=tf.constant_initializer(0.0))
    conv3 = tf.nn.conv2d(embeds_expanded,
                        W3,
                        strides=[1, 1, 1, 1],
                        padding='VALID',
                        name=f'conv-{f3}')
    conv_hidden3 = tf.nn.tanh(tf.add(conv3, b3), name=f'tanh-{f3}')
    # conv_hidden = tf.nn.relu(tf.add(conv, b), name=f'relu-{f3}')
    pool3 = tf.nn.max_pool(conv_hidden3,
                        ksize=[1, sequence_length - f3 + 1, 1, 1],
                        strides=[1, 1, 1, 1],
                        padding='VALID',
                        name=f'pool-{f3}')
    pooled_outputs.append(pool3)

In [9]:
with tf.name_scope(f'conv-maxpool-{f4}'):
    filter_shape4 = [f4, word_embed_size, 1, filter_num]
    W4 = tf.get_variable(f"W-{f4}", 
                        filter_shape4, 
                        initializer=tf.truncated_normal_initializer(stddev=0.1))
    b4 = tf.get_variable(f"b-{f4}", [filter_num], 
                        initializer=tf.constant_initializer(0.0))
    conv4 = tf.nn.conv2d(embeds_expanded,
                        W4,
                        strides=[1, 1, 1, 1],
                        padding='VALID',
                        name=f'conv-{f4}')
    conv_hidden4 = tf.nn.tanh(tf.add(conv4, b4), name=f'tanh-{f4}')
    # conv_hidden = tf.nn.relu(tf.add(conv, b), name=f'relu-{f4}')
    pool4 = tf.nn.max_pool(conv_hidden4,
                        ksize=[1, sequence_length - f4 + 1, 1, 1],
                        strides=[1, 1, 1, 1],
                        padding='VALID',
                        name=f'pool-{f4}')
    pooled_outputs.append(pool4)

In [11]:
embeds_expanded

<tf.Tensor 'ExpandDims:0' shape=(?, 20, 128, 1) dtype=float32>

In [10]:
for i in (W3, conv_hidden3, pool3):
    print(i)
    
print('---' * 10 )

for i in (W4, conv_hidden4, pool4):
    print(i)

Tensor("W-3/read:0", shape=(3, 128, 1, 64), dtype=float32)
Tensor("conv-maxpool-3/tanh-3:0", shape=(?, 18, 1, 64), dtype=float32)
Tensor("conv-maxpool-3/pool-3:0", shape=(?, 1, 1, 64), dtype=float32)
------------------------------
Tensor("W-4/read:0", shape=(4, 128, 1, 64), dtype=float32)
Tensor("conv-maxpool-4/tanh-4:0", shape=(?, 17, 1, 64), dtype=float32)
Tensor("conv-maxpool-4/pool-4:0", shape=(?, 1, 1, 64), dtype=float32)
